In [1]:
import csv
import pandas as pd
from owlready2 import *
import csv

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


# PreProcess

In [2]:
def edit_prospect_value(text):    
    
    if text.replace(",", "").isnumeric():
        return int(text.replace(",", ""))
    else:
        return sum( map(int, text.replace(",", "").replace("(N)", "").replace("(S)", "").split('/')))
    
regions_df_final = pd.read_csv('./data/regions_df_final.csv', index_col='Index')    
continents_df_final = pd.read_csv('./data/continents_df_final.csv', index_col='Index')    
continents_df_final['Population prospect by 2050'] = continents_df_final['Population prospect by 2050'].apply(lambda x: edit_prospect_value(x))

# Define Namespces

In [3]:
geo_ontology = get_ontology("http://ce.sharif.edu/onto")

ns_country = get_ontology("http://ce.sharif.edu/onto/country")
ns_region = get_ontology("http://ce.sharif.edu/onto/region")
ns_continent = get_ontology("http://ce.sharif.edu/onto/continent")
ns_lang = get_ontology("http://ce.sharif.edu/onto/language")

# Define Classes

In [4]:
with geo_ontology:
    class Territory(Thing): 
        pass
    class Country(Territory): 
        namespace = ns_country
        
    class Region(Territory): 
        namespace = ns_region

    class Continent(Territory): 
        namespace = ns_continent
        
    class Language(Thing):
        namespace = ns_lang
        
    AllDisjoint([Continent, Country, Region, Language])

# Object Properties


In [5]:
with geo_ontology:
    
    #Country
    class neighbourof(Country >> Country): 
        pass
    
    class country_has_region(Country >> Region, FunctionalProperty): 
        pass
    
    class hasLanguage(Country >> Language): 
        pass

    #Region
    class region_has_country(Region >> Country, InverseFunctionalProperty): 
        inverse_property = country_has_region
        
    class region_has_subregion(Region >> Region, InverseFunctionalProperty):
        pass

    class subregion_has_region(Region >> Region, InverseFunctionalProperty):
        inverse_property = region_has_subregion
    
    class region_has_continent(Region >> Continent, FunctionalProperty): 
        pass
    
    
    #Continent
    class continent_has_region(Continent >> Region): 
        inverse_property = region_has_continent


# Data Type Properties


In [6]:
with geo_ontology:
    
    #Teritory
    class tr_name(Territory >> str, FunctionalProperty): pass
    class tr_population(Territory >> int, FunctionalProperty): pass
    class tr_area(Territory >> int, FunctionalProperty): pass
    class tr_pupulation_prospect(Continent >> int, FunctionalProperty): pass
    

# Insert Data

In [7]:
def convertURI(text):
    return text.replace("&", "and").replace(" ", "_").replace("-", "_")

In [8]:
with geo_ontology:
    for index, row in continents_df_final.iterrows():
        the_continent = Continent(convertURI(row['Index.1']), ns_continent, tr_name=row['Index.1']) 
        if row['Surface area(km2)']: the_continent.tr_area = int(row['Surface area(km2)'])
        if row['Population(000, 2020)']: the_continent.tr_population = int(row['Population(000, 2020)'])
        if row['Population prospect by 2050']: the_continent.tr_pupulation_prospect = int(row['Population prospect by 2050'])

        if row['Regions']:
            for i in row['Regions'].replace("[","").replace("]", "").replace("'","").split(", "):
                the_region = Region(convertURI(i), ns_region) 
                the_continent.continent_has_region.append(the_region)

In [10]:
with geo_ontology:
    for index, row in regions_df_final.iterrows():
        the_region = Region(convertURI(row['Index.1']), ns_region, tr_name=row['Index.1']) 
        if row['Surface area(km2)']: the_region.tr_area = int(row['Surface area(km2)'])
        if row['Population(000, 2020)']: the_region.tr_population = int(row['Population(000, 2020)'])
            
        if row['Region'] in ['Sub-Saharan Africa', 'Latin America & Caribbean']:
            the_super_region = Region(convertURI(row['Region']), ns_region) 
            the_region.subregion_has_region.append(the_super_region)
            
        if row['Countries']:
            for i in str(row['Countries']).replace("[","").replace("]", "").replace("'","").split(", "):
                the_country = Country(convertURI(i), ns_country) 
                the_region.region_has_country.append(the_country)   

In [11]:
geo_ontology.save(file="./output/ontology.owl")